In [54]:
import numpy as np

In [55]:
# Define job class
class Job:
    def __init__(self, mach_req, proc_time, number):
        self.mach_req = mach_req  # Machines required for the job
        self.proc_time = proc_time  # Processing times for the job on each machine
        self.processes = [i + (number-1)*len(mach_req) for i in range(1,len(mach_req)+1)]
        self.stage = [False] * len(mach_req)  # Stages of the job completed on machines
        self.counter = 0  # Index to track the current machine for the job
        self.mach_needed = self.mach_req[0]  # Current machine needed for the job
        self.number = number  # Job number
        self.free_time = 0  # Time when the job becomes available for scheduling
        self.job_done = False  # Flag to indicate if the job is completed

    def __lt__(self, other):
        # Comparison method to determine if this job has a shorter processing time
        return self.proc_time[self.counter] < other.proc_time[other.counter]
    
    def __add__(self, other):
        # Add the job's processing time to another object's free time
        return self.proc_time[self.counter] + other.free_time
    
    def __le__(self, other):
        # Comparison method to determine if this job can be scheduled on a machine
        return self.free_time <= other.free_time
    
    def __bool__(self):
        # Boolean representation of a job indicating if it's completed
        return self.job_done
    

# Define machine class
class Machine:
    def __init__(self, number):
        self.free = True  # Flag to indicate if the machine is free
        self.free_time = 0  # Time when the machine becomes available for scheduling
        self.number = number  # Machine number

    def __lt__(self, other):
        # Comparison method to determine if this machine has a shorter free time
        return self.free_time < other.free_time
    
    def __le__(self, other):
        # Comparison method to determine if this machine has an earlier or equal free time
        return self.free_time <= other.free_time

# Function to find the shortest job available for a machine
def next_process(job_next: Job, mach: Machine, process_num: int):
    # Get the number of the machine that is free
    mach_num = mach.number
    process = process_num%len(job_next.processes)

    print(process_num, not all(job_next.stage[0:process-1]))
    if not all(job_next.stage[0:process-1]):
        return None
    
    if job_next.free_time != mach.free_time:
        # Synchronize the job's free time with the machine's free time
        time = max(job_next.free_time, mach.free_time)
        job_next.free_time = time
        mach.free_time = time
        
    processing_time = job_next.proc_time[job_next.counter]

    # Mark the job as done for this stage
    job_next.stage[job_next.counter] = True
    job_next.counter += 1
    
    if all(job_next.stage):
        # If all stages of the job are completed, mark it as done
        job_next.job_done = True

    return job_next, processing_time

# Main constructive scheduling function
def makespan_solution(jobs, machines, solution):
    m = len(machines); n = len(jobs); pending_jobs = []

    while not all(jobs):   
        for j in range(n):
            for i in range(m):
                if all(jobs):
                    return max([mach.free_time for mach in machines])
                 
                if pending_jobs != []:
                    for pending in pending_jobs:
                        #print("pending", pending)
                        job_next = jobs[int(np.ceil(pending[0]/m) -1)]
                        mach = pending[1]
                        aux = next_process(job_next,  mach, pending[0])

                        if aux is not None:
                            job_next, processing_time = aux[0], aux[1]
                            pending_jobs.remove(pending)
                            job_next.free_time += processing_time
                            mach.free_time += processing_time
                        else:
                            print("skip")
                            continue
                        
                # Find the machine with the earliest free time
                job_next = jobs[int(np.ceil(solution[i][j]/m) -1)]
                mach = machines[i]
                
                aux = next_process(job_next,  machines[i], solution[i][j])
                
                if aux is not None:
                     job_next, processing_time = aux[0], aux[1]
                    # Update the machine's free time and the job's free time
                    job_next.free_time += processing_time
                    mach.free_time += processing_time
                else:
                    #print(pending_jobs)
                    pending_jobs.append((solution[i][j], machines[i]))
                    continue
            
            
        

In [56]:
solution = [[168, 108, 141, 154, 51, 39, 220, 26, 103, 188, 14, 210, 74, 125, 87],
            [31, 216, 97, 203, 177, 113, 146, 71, 59, 15, 85, 191, 126, 162, 29],
            [166, 139, 47, 67, 6, 184, 101, 40, 223, 114, 209, 84, 127, 165, 30],
            [92, 77, 5, 143, 185, 222, 27, 178, 44, 208, 161, 117, 60, 75, 135],
            [16, 198, 151, 3, 65, 215, 99, 53, 81, 43, 180, 149, 116, 190, 130],
            [76, 181, 17, 107, 46, 200, 155, 173, 100, 38, 70, 13, 225, 150, 128],
            [136, 1, 63, 170, 214, 96, 49, 35, 201, 24, 186, 157, 123, 119, 89],
            [61, 93, 18, 109, 152, 4, 172, 145, 221, 54, 41, 205, 83, 194, 132],
            [196, 32, 212, 94, 62, 182, 140, 21, 174, 9, 158, 57, 115, 90, 131],
            [138, 33, 66, 48, 23, 175, 202, 11, 187, 224, 105, 118, 88, 164, 133],
            [211, 121, 64, 171, 183, 50, 7, 144, 25, 102, 42, 160, 207, 86, 120],
            [106, 137, 122, 98, 22, 36, 8, 219, 80, 56, 206, 179, 73, 192, 163],
            [91, 2, 169, 213, 34, 110, 78, 156, 69, 204, 28, 147, 58, 193, 129],
            [199, 20, 153, 142, 111, 217, 37, 52, 176, 79, 12, 104, 189, 72, 134],
            [197, 167, 95, 19, 68, 218, 10, 112, 55, 159, 82, 148, 45, 124, 195]]

In [57]:
import os
import numpy as np
directory_path = 'JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

#Reading all the information
for file in directory_files:
  with open(directory_path + "/" + file, "r" ) as f:
  #solution = [[11,4,9,2],[10,8,6,3],[7,1,5,12]]
  #with open("AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = max([sum([x[1] for x in machines[i]]) for i in machines])

  jobs = [Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines = [Machine(i+1) for i in range(m)]

  makespan_solution(jobs,machines, solution)
  print(max([x.free_time for x in machines]))

  break

168 True
168 True
skip
31 False
168 True
skip
166 False
168 True
skip
92 True
168 True
skip
92 True
skip
16 False
168 True
skip
92 True
skip
76 False
168 True
skip
92 True
skip
136 False
168 True
skip
92 True
skip
61 False
168 True
skip
92 True
skip
196 False
168 True
skip
92 True
skip
138 True
168 True
skip
92 True
skip
138 True
skip
211 False
168 True
skip
92 True
skip
138 True
skip
106 False
168 True
skip
92 True
skip
138 True
skip
91 False
168 True
skip
92 False
199 True
168 True
skip
138 True
skip
199 True
skip
197 False
168 True
skip
138 True
skip
199 True
skip
108 True
168 True
skip
138 True
skip
199 True
skip
108 True
skip
216 True
168 True
skip
138 True
skip
199 True
skip
108 True
skip
216 True
skip
139 True
168 True
skip
138 True
skip
199 True
skip
108 True
skip
216 True
skip
139 True
skip
77 False
168 True
skip
138 True
skip
199 True
skip
108 True
skip
216 True
skip
139 True
skip
198 False
168 True
skip
138 True
skip
199 False
216 True
skip
139 True
skip
181 False
168 True
s

IndexError: index 15 is out of bounds for axis 0 with size 15

In [64]:
all(jobs[3].stage)

True

In [38]:
solution = [[4,11,9,2],[10,8,6,3],[7,1,5,12]]

#with open(directory_path + "/" + file, "r" ) as f:
with open("AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]

jobs = [Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

#Sorted array in which the first value is always the first machine that 
machines = [Machine(i+1) for i in range(m)]

makespan_solution(jobs,machines, solution)
print(max([x.free_time for x in machines]))

4 False
10 False
7 False
11 False
8 False
1 False
9 False
6 True
pending (6, <__main__.Machine object at 0x7ffb7ff3b9d0>)
5 False
2 False
3 False
12 False
51


# Inaportante

In [20]:
machines[0].free_time

39

In [ ]:
#with open(directory_path + "/" + file, "r" ) as f:
with open("AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]

machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
cota_inferior = max([sum([x[1] for x in machines[i]]) for i in machines])

#auxiliary variables
jLen = range(n); mLen = range(m); process = 1

# Define jobs and machines and each process
#jobs = {job_num: [process, machine, time]} | machines = {machine_num: [process, job, time]} | processes = {process_num: [job, machine, time]}
processes = dict(); jobs = {i:[] for i in range(1,n+1)}; machines = {j:[] for j in range(1,m+1)}

for j in jLen:
    job = j+1; jobs[job] = []
    for i in mLen:
        jobs[job].append([process, machines_required[j][i],processing_time[j][i]])
        processes[process] = [job, machines_required[j][i],processing_time[j][i]]
        machines[machines_required[j][i]].append([process, job, processing_time[j][i]])
        process += 1

pheromones = {j:[] for j in range(1,m+1)}
for j in mLen:
    pheromones[j+1] = []

In [4]:
# Define pheromones
for j in mLen:
    for i in jLen:
        pheromones[j+1].append([i+1, 1])

# Define ants
ants = {i:[] for i in range(1,n+1)}
for i in range(1,n+1):
    ants[i] = [i]

# Define parameters
alpha = 1; beta = 1; rho = 0.5; Q = 1; t_max = 1000; t = 0

# Define best solution
best_solution = []; best_solution_time = 0

# Define initial solution
for i in range(1,n+1):
    for j in range(1,m+1):
        if machines_required[i-1][j-1] == 1:
            best_solution.append([i,j,processing_time[i-1][j-1]])
            best_solution_time += processing_time[i-1][j-1]
            break

# Define initial pheromones
for j in mLen:
    for i in jLen:
        pheromones[j+1][i][1] = 1/best_solution_time

# Define initial probabilities
probabilities = {j:[] for j in range(1,m+1)}
for j in mLen:
    probabilities[j+1] = []

# Define probabilities
for j in mLen:
    for i in jLen:
        probabilities[j+1].append([i+1, pheromones[j+1][i][1]])

# Define initial probabilities
for j in mLen:
    for i in jLen:
        probabilities[j+1][i][1] = pheromones[j+1][i][1]/sum([x[1] for x in probabilities[j+1]])



In [6]:
processes

{1: [1, 3, 12],
 2: [1, 1, 10],
 3: [1, 2, 12],
 4: [2, 1, 6],
 5: [2, 3, 8],
 6: [2, 2, 6],
 7: [3, 3, 4],
 8: [3, 2, 9],
 9: [3, 1, 4],
 10: [4, 2, 15],
 11: [4, 1, 10],
 12: [4, 3, 13]}

In [36]:
#def solution_time(solution, processes):
solution = [[2,4,3,1],[4,3,2,1],[3,1,2,4]]
solution = [[4,11,9,2],[10,8,6,3],[7,1,5,12]]
machines = {j+1:[processes[solution[j][i]][2] for i in range(n)] for j in range(m)}
machines = {j+1:solution[j] for j in range(m)}
machines

machines_free_time = [processes[solution[j][0]][2] for j in range(m)]
processes_done = {i+1: solution[i][0] for i in range(m)}




In [37]:
machines_free_time, processes_done

([6, 15, 4], {1: 4, 2: 10, 3: 7})

In [41]:
a = [1,1,1,3,4,2]
not all(a[0:0])

False

In [44]:
import random

# Define the cities and their coordinates
cities = {
    "A": (0, 0),
    "B": (2, 4),
    "C": (5, 2),
    "D": (7, 6),
    "E": (1, 8),
}

# Parameters
num_ants = 10
num_iterations = 50
alpha = 1.0  # Pheromone importance
beta = 2.0   # Heuristic importance
rho = 0.5    # Pheromone evaporation rate

# Initialize pheromone levels on each edge
pheromone = {}
for city1 in cities:
    for city2 in cities:
        if city1 != city2:
            pheromone[(city1, city2)] = 1.0

# Define a function to calculate the distance between two cities
def calculate_distance(city1, city2):
    x1, y1 = cities[city1]
    x2, y2 = cities[city2]
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Ant construction of a tour
def construct_tour(pheromone, alpha, beta):
    tour = []
    unvisited_cities = list(cities.keys())
    current_city = random.choice(unvisited_cities)
    unvisited_cities.remove(current_city)

    while unvisited_cities:
        probabilities = []
        for city in unvisited_cities:
            pheromone_level = pheromone[(current_city, city)]
            distance = calculate_distance(current_city, city)
            probability = (pheromone_level ** alpha) * ((1.0 / distance) ** beta)
            probabilities.append((city, probability))

        # Select the next city based on the calculated probabilities
        total_probability = sum(prob for _, prob in probabilities)
        threshold = random.uniform(0, total_probability)
        cum_probability = 0
        for city, prob in probabilities:
            cum_probability += prob
            if cum_probability >= threshold:
                next_city = city
                break

        tour.append((current_city, next_city))
        current_city = next_city
        unvisited_cities.remove(current_city)

    # Add the last edge to complete the tour
    tour.append((current_city, tour[0][0]))
    return tour

# Main ACO loop
for iteration in range(num_iterations):
    ant_tours = []

    # Construct tours for each ant
    for ant in range(num_ants):
        tour = construct_tour(pheromone, alpha, beta)
        ant_tours.append(tour)

    # Update pheromone levels
    for city1, city2 in pheromone.keys():
        pheromone[(city1, city2)] *= (1 - rho)

    for tour in ant_tours:
        tour_length = sum(calculate_distance(city1, city2) for city1, city2 in tour)
        for city1, city2 in tour:
            pheromone[(city1, city2)] += 1.0 / tour_length

# Find the best tour
best_tour = min(ant_tours, key=lambda x: sum(calculate_distance(city1, city2) for city1, city2 in x))
best_tour_length = sum(calculate_distance(city1, city2) for city1, city2 in best_tour)

print("Best Tour:", best_tour)
print("Best Tour Length:", best_tour_length)


Best Tour: [('D', 'E'), ('E', 'B'), ('B', 'A'), ('A', 'C'), ('C', 'D')]
Best Tour Length: 24.77709766308808
